# Time Series Model Validation Full Suite

## 1. Introduction

The Time Series Data Validation Full Suite notebook aims to demonstrate the application of various data validation tests using the **ValidMind MRM Platform** and **Developer Framework**. 

In this demo, we will use the `time_series_model_validation` **test suite** to run multiple model validation metrics on several pre-trained time series models.  



## 2. Setup 

Prepare the environment for our analysis. First, **import** all necessary libraries and modules required for our analysis. Next, **connect** to the ValidMind MRM platform, which provides a comprehensive suite of tools and services for model validation.

Finally, define and **configure** the specific use case we are working on by setting up any required parameters, data sources, or other settings that will be used throughout the analysis. 

### Import Libraries

In [ ]:
# Load API key and secret from environment variables
%load_ext dotenv
%dotenv .env

# ValidMind libraries
import validmind as vm

### Connect to the ValidMind Library

In [ ]:

import validmind as vm

vm.init(  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  project = "clhhzo21s006wl9rl0swhv40h")


### Find All Test Suites and Plans Available in the Developer Framework

We can find all the **test suites** and **test plans** available in the developer framework by calling the following functions:

- All test suites: `vm.test_suites.list_suites()`
- All test plans: `vm.test_suites.list_plans()`
- Describe a test plan: `vm.test_suites.describe_plan("time_series_data_quality")`
- List all available tests: `vm.tests.list_tests()`

In [ ]:
vm.test_suites.list_suites()

In [ ]:
vm.test_suites.list_plans()

## 3. Load Models

#### Load Pre-Trained Time Series Models

In [ ]:
# Currently only fred pre-trained models are available
from validmind.datasets.regression import fred as demo_dataset
model_A, train_df_A, test_df_A = demo_dataset.load_model('fred_loan_rates_model_3')
model_B, train_df_B, test_df_B = demo_dataset.load_model('fred_loan_rates_model_4')

#### Create ValidMind Datasets

In [ ]:
# Initialize training and testing datasets for model A
vm_train_ds_A = vm.init_dataset(dataset=train_df_A, type="generic", target_column=demo_dataset.target_column)
vm_test_ds_A = vm.init_dataset(dataset=test_df_A, type="generic", target_column=demo_dataset.target_column)

# Initialize training and testing datasets for model B
vm_train_ds_B = vm.init_dataset(dataset=train_df_B, type="generic", target_column=demo_dataset.target_column)
vm_test_ds_B = vm.init_dataset(dataset=test_df_B, type="generic", target_column=demo_dataset.target_column)

#### Create ValidMind Models

In [ ]:
# Initialize model A
vm_model_A = vm.init_model(
    model = model_A,
    train_ds=vm_train_ds_A,
    test_ds=vm_test_ds_A)

# Initialize model B
vm_model_B = vm.init_model(
    model = model_B,
    train_ds=vm_train_ds_B,
    test_ds=vm_test_ds_B)


list_of_models = [vm_model_A, vm_model_B]

## 4. Model Validation

### User Configuration of Test Suite 

Users can input the configuration to a test suite using **`config`**, allowing fine-tuning the suite according to their specific data requirements. 

**Time Series Forecast params**
- `transformation` specify the desired plotting settings for regression forecast evaluation. In this particular configuration, the chosen transformation is "integrate."


In [ ]:
config= {
    "regression_forecast_plot_levels": {
        "transformation": "integrate",
    },
    "regression_sensitivity_plot": {
        "transformation": "integrate",
        "shocks": [0.1, 0.2],
    }
}

In [ ]:

full_suite = vm.run_test_suite(
    "time_series_model_validation",
    model = vm_model_B,
    models = list_of_models,
    config = config,
)